In [1]:
from scipy import integrate
import sympy as sp
import numpy as np
from numpy import linalg as la 
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML
font = {'size'   : 15}
matplotlib.rc('font', **font)
from New_Homogenized_system_coefficients1_pwc import C_values, Homogenized_system_coef
fft = np.fft.fft
ifft = np.fft.ifft
from tqdm import tqdm 
import os

In [ ]:
def rk3(v,xi,rhs,dt):
    y2 = v + dt*rhs(v,xi)
    y3 = 0.75*v + 0.25*(y2 + dt*rhs(y2,xi))
    v_new = 1./3 * v + 2./3 * (y3 + dt*rhs(y3,xi))
    return v_new

In [ ]:
num_plots =  450

In [ ]:
def solve_isothermal_gas_equations(m,dt, num_plots):
    L = 1000 
    x = np.arange(-m/2, m/2) * (L/m)
    xi = np.fft.fftfreq(m) * m * 2 * np.pi / L

    if cross_section == 'piecewise':
        Amp = 1/20
        rhobar_initial = Amp*np.exp(-1*(x/8)**2)
    elif cross_section == 'variable':
        Amp = 1/12 
        rhobar_initial =  Amp * np.exp(-(x/5)**2) 
    elif cross_section == 'pwc_small':
        Amp = 1/40
        rhobar_initial =  Amp*np.exp(-1*(x/10)**2)
    elif cross_section == 'constant':
        Amp = 1/200
        rhobar_initial = Amp*np.exp(-0.36*(x)**2)
        
    q0 = np.zeros_like(rhobar_initial)
    drhodx = np.zeros_like(rhobar_initial)
    dqdx = np.zeros_like(rhobar_initial)
    drhodxt = np.real(ifft(1j * xi * fft(rhobar_initial)))
    dqdxt = np.real(ifft(1j * xi * fft(q0)))
    v = np.zeros((2, len(x)))
    v[0, :] = rhobar_initial; v[1, :] = q0
    vhat = np.zeros_like(v)
    vhat[0, :] = drhodxt; vhat[1, :] = dqdxt
    tmax = 900
    

    def rhs(v, xi):
        d = np.zeros_like(v)
        rho = v[0, :]
        q = v[1, :]
        q_hat = fft(q)
        rho_hat = fft(rho)
        drhodx = np.real(ifft(1j * xi * rho_hat))
        drho2dx2 = np.real(ifft((1j * xi)**2 * rho_hat))
        drho3dx3 = np.real(ifft((1j * xi)**3 * rho_hat))
        dqdx  = np.real(ifft(1j * xi * q_hat))
        dq2dx2 = np.real(ifft((1j * xi)**2 * q_hat))

        inv1 = 1 / (1 - r5b * (1j * xi)**2)
        rhs1 = r1*dqdx + r2*dq2dx2 + r3*q*drhodx + r4*dqdx*q**2 + r6*q*rho*drhodx + r7*dqdx*drhodx + r8*q*drho2dx2
        dqdt = np.real(ifft(inv1 * fft(rhs1)))

        inv2 = 1 / (1 - beta11b * (1j * xi)**2)
        rhs2 = beta1*drhodx + beta2*q*dqdx + beta3*rho*drhodx + beta4*drho2dx2 + beta5*q*rho*dqdx + beta6*dqdx**2\
             + beta7*q*dq2dx2 + beta8*drhodx*q**2 + beta9*drhodx**2 + beta10*rho*drho2dx2  
        drhodt = np.real(ifft(inv2 * fft(rhs2)))

        d[0, :] = dqdt
        d[1, :] = drhodt
        return d

    nplt = np.floor((tmax / num_plots) / dt)
    nmax = int(round(tmax / dt))
    
    frames = [v.copy()]
    tt = [0]
    progress_bar = tqdm(total=nmax)
    for n in range(1, nmax + 1):
        v_new = rk3(v, xi, rhs, dt)
        v = v_new.copy()
        t = n * dt
        if n % nplt == 0:
            frames.append(v.copy())
            tt.append(t)
            progress_bar.update(nplt)
    progress_bar.close()
    return frames, x, tt, xi

def plot_solution(frames, x, tt, xi):
    fig = plt.figure(figsize=(12, 10))
    axes = fig.add_subplot(211)
    axes2 = fig.add_subplot(212)
    line, = axes.plot(x, frames[0][0, :], lw=3)
    line2, = axes2.plot(x, frames[0][1, :], lw=3)
    xi_max = np.max(np.abs(xi))
    axes2.set_xlabel(r'$x$', fontsize=30)
    axes.set_ylabel(r'$\rho$', fontsize=30)
    axes2.set_ylabel(r'$q$', fontsize=30)
    plt.close()

    def plot_frame(i):
        line.set_data(x, frames[i][0, :])
        line2.set_data(x, frames[i][1, :])
        axes.set_title('t= %.2e' % tt[i])
        axes.set_xlim((x[0], x[-1]))
        axes2.set_xlim((x[0], x[-1]))

    anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                              frames=len(frames), interval=100,
                                              repeat=False)
    return HTML(anim.to_jshtml())


In [ ]:
cross_section = 'piecewise'
if cross_section == 'pwc_small':
    a = lambda y: np.piecewise(y, [y- np.floor(y) < 0.5, y- np.floor(y) >= 0.5], [1/2, 3/4])
    dady = lambda y: np.zeros_like(y)
    m=6000

elif cross_section == 'variable':
    a = lambda y: (3/5) + (2/5) *np.sin(2.*np.pi*y)
    dady = lambda y: (2/5)*2*np.pi*np.cos(2.*np.pi*y)
    m = 6000

elif cross_section == 'piecewise':
    a = lambda y: np.piecewise(y, [y- np.floor(y) < 0.5, y- np.floor(y) >= 0.5], [1/4, 3/4])
    dady = lambda y: np.zeros_like(y)
    m = 6000
elif cross_section == 'constant':
    a = lambda y: 5
    dady = lambda y: 0
    m = 800

In [ ]:
gamma = 1.4 
kappa =1
p_0 = 0.3  # Corresponds to \(\rho_0\) (rho zero) in the paper
P1 = gamma*kappa*p_0**(gamma-1) #first derivative of the pressure
P11= (gamma-1)*gamma*kappa*p_0**(gamma-2) #second derivative of the pressure


In [ ]:
print("P1 =", P1)
print("P11 =", P11)

In [ ]:
delta = 1
C1, C2, C3, C4, C5, C6, C7, C8, C9,C10,C11,C12,C13,C14,C15,C16,avga,ainvavg = C_values(a=a,dady=dady,p_0=p_0,delta=delta)

In [ ]:
r1, r2, r3, r4, r5,r5b, r6,r7,r8, beta1, beta2, beta3, beta4, beta5, beta6, beta7,beta8, beta9, beta10, beta11,beta11b = Homogenized_system_coef(C1, C2, C3, C4, C5, C6, C7, C8, C9,C10,C11,C12,C13,C14,C15,C16,avga,ainvavg, p_0, P1, P11, delta)

In [ ]:
print("<a>^2= ",avga**2)
print('C1= ',C1)
print('C2= ',C2)
print('C3= ',C3)
print('C4= ',C4)
print('C5= ',C5)
print('C6= ',C6)
print('C7= ',C7)
print('C8= ',C8)
print('C9= ',C9)
print('C10= ',C10)
print('C11= ',C11)
print('C12= ',C12)
print('C13= ',C13)
print('C14= ',C14)
print('C15= ',C15)
print('C16= ',C16)
avga

In [ ]:
print("r1= ",r1)
print("r2= " , r2)
print("r3= " , r3)
print("r4= " , r4)
print("r5= " , r5)
print("r5b= " , r5b)
print("r6= " , r6)
print("r7= " , r7)
print("r8= " , r8)


print("\u03B21= " ,beta1)
print("\u03B22= " ,beta2)
print("\u03B23= " ,beta3)
print("\u03B24= " ,beta4)
print("\u03B25= " ,beta5)
print("\u03B26= " ,beta6)
print("\u03B27= " ,beta7)
print("\u03B28= " ,beta8)
print("\u03B29= " ,beta9)
print("\u03B210= " ,beta10)
print("\u03B211= " ,beta11)
print("\u03B211b= " ,beta11b)

print(1/avga)

In [ ]:
dt = 9e-05 #1.73/(m/2)**2
print(dt)
frames, x, tt, xi=solve_isothermal_gas_equations(m,dt,num_plots)

In [ ]:
plot_solution(frames, x, tt, xi)

In [ ]:
from clawpack import pyclaw

# Define the path
path = "/Users/busalels/Desktop/KAUST/fall2024/research/Homogenization_of_isothermal_gas_equation/Capacity_function_pwc/_output"

# Define the function to read a frame
def FV_solution(i):
    frame = pyclaw.Solution(i, file_format='petsc', file_prefix='claw', read_aux=True, path=path)
    return frame

#Initialize an empty list to store the data
FV_frames = []

# Loop over the range and append data to the list
for i in range(num_plots):
    frame = FV_solution(i)
    FV_frames.append(frame.q)  

# FV_frames contain all the frame.q data
xFV = frame.domain.grid.x.centers

In [ ]:
a = frame.aux[0:]

In [ ]:
rho_constant = 0.3
def plot_solution(frames,x,xFV,FV_frames, tt,a):
    fig = plt.figure(figsize=(10,10))
    axes = fig.add_subplot(211)
    axes2 = fig.add_subplot(212)
    line, = axes.plot(x,frames[0][0,:]+rho_constant,lw=3,  label='Homogenized')
    line2, = axes2.plot(x,frames[0][1,:],lw=2)
    lineFV, = axes.plot(xFV,FV_frames[0][0],lw=3, label ='FV')
    line2FV, = axes2.plot(xFV, a.flatten()*FV_frames[0][1].flatten(), lw=3)
    axes.set_xlabel(r'$x$',fontsize=30)
    axes2.set_xlabel(r'$x$',fontsize=30)
    axes.set_ylabel(r'$\rho$',fontsize=30)
    axes2.set_ylabel(r'$q$',fontsize=30)
    #plt.tight_layout()
    plt.close()


    def plot_frame(i):
        line.set_data(x,frames[i][0,:]+rho_constant)
        line2.set_data(x,frames[i][1,:])
        lineFV.set_data(xFV,FV_frames[i][0])
        line2FV.set_data(xFV, a.flatten()*FV_frames[i][1].flatten())
        axes.set_xlim((0,x[-1]))
        axes2.set_xlim((0,x[-1]))
        
    axes.legend()

    anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                              frames=len(FV_frames), interval=100,
                                              repeat=False)
    return HTML(anim.to_jshtml())


In [ ]:
plot_solution(frames,x,xFV,FV_frames, tt,a)